In [1]:
import fitz  # PyMuPDF
import re
import pandas as pd
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.embeddings import HuggingFaceEmbeddings
import os

In [2]:
def read_pdf(file_path):
    text = ""
    with fitz.open(file_path) as pdf_document:
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            text += page.get_text()

    return text 

In [3]:
folder_path = "/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/train_book"
file_names = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        file_names.append(file_path)

In [4]:
len(file_names)

10

In [ ]:
'''
full_text = ""
for file_path in file_names:
    full_text += read_pdf(file_path) + "\n"
    
with open("/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/text/full_text.txt", 'w') as file:
    file.write(full_text)
'''

In [5]:
title_names = []
for filename in os.listdir(folder_path):
    title_names.append(filename.rstrip(".pdf"))

In [6]:
len(title_names)

11

In [ ]:
'''
path = "/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/text/"
for i in range(len(title_names)):
    full_text = read_pdf(file_names[i])
    with open(path + title_names[i] + ".txt", 'w') as file:
        file.write(full_text)
'''

In [7]:
title_names = title_names[1:]

In [8]:
len(title_names)

10

In [9]:
#path = "/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/text/"
doc_text = []
for i in range(len(title_names)):
    full_text = read_pdf(file_names[i])
    doc_text.append(full_text)

In [ ]:
'''
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_text(doc_text)
'''

In [11]:
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)

In [12]:
titles = []
texts = []
for i in range(len(doc_text)):
    chunk = text_splitter.split_text(doc_text[i])
    num = len(chunk)
    #print(num)
    for k in range(num):
        titles.append(title_names[i])
        texts.append(chunk[k] + "\t" + title_names[i])

In [13]:
len(texts)

6637

In [ ]:
'''
documents = []
for i in range(len(file_names)):
    loader = PyPDFLoader(file_names[i])
    documents.extend(loader.load())
'''

In [ ]:
'''
with open('/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/text/chunk.txt', 'w') as file:
    for i in range(len(texts)):
        file.write(texts[i] + "<endofchunk>")
'''

In [14]:
# all-mpnet-base-v2
# vietnamese-sbert
# BAAI/bge-m3
embedding_function = SentenceTransformerEmbeddings(model_name="qminh369/rag-book")
#embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-m3")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.56k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [15]:
embedding_function

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
), model_name='qminh369/rag-book', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [16]:
# load it into Chroma
#db = Chroma.from_documents(documents, embedding_function, persist_directory="/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/chroma_db")
#db = Chroma.from_texts(texts, embedding_function, persist_directory="/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/chroma_db")
#db = Chroma.from_texts(texts, embedding_function, persist_directory="/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/db/chroma_db_ver1")
#db = Chroma.from_texts(texts, embedding_function, persist_directory="/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/db/chroma_db_bge")
db = Chroma.from_texts(texts, embedding_function, persist_directory="/home/skmlab1/data/data/QuangMinh/Learning/Clone/RAGBOOK/note/db/chroma_db_finetune")

In [17]:
db